In [1]:
import os
from datetime import datetime
# Make logging folder named according to current time
start_time_str = datetime.today().strftime('%d_%m_%Y__%H_%M_%S')
log_dir = './logs/'f"log_{start_time_str}"
os.makedirs(log_dir)

# Initialize logger that adds to text file in current logging folder
from logging_utils import *

init_logger(save_dir=log_dir)
logger = get_logger()




In [2]:
import pathlib
import subprocess
# Get the absolute path to your repository, 
# no matter where you are running this code from

repo_path = os.getcwd() 


git_branch = subprocess.check_output(
    ["git", "-C", repo_path, "rev-parse", "--abbrev-ref", "HEAD"]).strip().decode('UTF-8')

git_commit_short_hash = subprocess.check_output(
    ["git", "-C", repo_path, "describe", "--always"]).strip().decode('UTF-8')


In [3]:
import numpy as np

dim=500
x0=np.squeeze(np.zeros((dim,1)))
x0Type='Zero Vector'

FD_StencilMat=np.zeros((dim,dim))
b = np.ones(dim-1)
c = -2*np.ones(dim)
np.fill_diagonal(FD_StencilMat[1:], b)
np.fill_diagonal(FD_StencilMat[:,1:], b)
np.fill_diagonal(FD_StencilMat, c)
A=FD_StencilMat
AType='1D Laplacian'


retrain_freq=3
e1 = 1e-8
e2 = 1e-10
nmax_iter = 50
restart   = 64

sigma=0.02
DomainL=-5.0
DomainR=5.0



n_steps =1000
InputDim=dim
HiddenDim=dim
HiddenDim2=dim
OutputDim=dim

In [4]:
# Initial log message
logger.info("Benchmark 500 dim problem. 2 Layer linear Conv Net. The network is set up so we start with 1 channel, then go to 'dim' number of channels with 1-dim out put. This is then flattned and used as the linear output. Relu activation ")

#  The source is a narrow gaussian that depends on the integer 'time' in a nonlinear way. The gaussian is on the interval [-5,5], and constrained to move on [-3,3]. ")

2020-05-17 11:22:40 [INFO] <ipython-input-4-06a642bf41d0> 2 - Benchmark 500 dim problem. 2 Layer linear Conv Net. The network is set up so we start with 1 channel, then go to 'dim' number of channels with 1-dim out put. This is then flattned and used as the linear output. Relu activation 


In [5]:
# Set up parameter class
class MyParamsClass():
    def __init__(self, AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash):
        self.AType = AType
        self.dim = dim
        self.nmax_iter = nmax_iter
        self.restart = restart
        self.n_steps =n_steps
        self.retrain_freq=retrain_freq
        self.e1=e1
        self.e2=e2
        self.x0Type=x0Type
        self.InputDim=InputDim
        self.HiddenDim=HiddenDim
        self.HiddenDim2=HiddenDim2
        self.OutputDim=OutputDim
        self.sigma=sigma
        self.DomainL=DomainL
        self.DomainR=DomainR
        self.git_branch=git_branch
        self.git_commit_short_hash=git_commit_short_hash

In [6]:
import os
import json

params = MyParamsClass(AType,dim,nmax_iter,restart,retrain_freq,e1,e2,x0Type,n_steps,InputDim,HiddenDim,HiddenDim2,OutputDim,sigma,DomainL,DomainR,git_branch,git_commit_short_hash)
sorted_params_dict = {k: params.__dict__[k] for k 
                      in sorted(params.__dict__.keys())}

params_filepath = log_dir+'/params.json'
json.dump(sorted_params_dict, open(params_filepath, 'w'), indent=4)

In [7]:
from src_dir import *

In [8]:
# @nn_preconditioner(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
# def MLGMRES(A, b, x0, e, nmax_iter,IterErr0_sum,IterErr0,ProbCount,Add,restart,debug):
#     return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [9]:
@timer
@nn_preconditioner_timed(retrain_freq=retrain_freq, debug=True,InputDim=InputDim,HiddenDim=HiddenDim,HiddenDim2=HiddenDim2,OutputDim=OutputDim)
def MLGMRES(A, b, x0, e, nmax_iter,Err_list,IterErr0_AVG,ProbCount,restart,debug,refine):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)
    

In [10]:
@timer
def GMRES_timed(A, b, x0, e, nmax_iter,restart,debug):
    return GMRES(A, b, x0, e, nmax_iter,restart, debug)

In [ ]:
Err_list=[]
IterErr0_AVG=0.0

debug=True

refine1=False
refine2=True

run_time=0.0
run_time_ML=0.0
trainTime_total=0.0
forwardTime_Total=0.0
run_time_ML_NoAdd=0.0


x=np.linspace(DomainL,DomainR,dim)



for ProbIdx in range(n_steps):

    loc=3.0*np.sin(0.5*ProbIdx*np.abs(np.cos(0.5*ProbIdx)))
    b=Gauss_pdf(x,loc,sigma)
    b_norm=np.linalg.norm(b)



    OutList1,run_time1_ML=MLGMRES(A, b/b_norm, x0, e1, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine1)  # normalize b for optimal NN performance.
    Out=OutList1[0]
    Err_list=OutList1[1]
    IterErr0_AVG=OutList1[2]
    forwardTime1=OutList1[4]


    OutList2,run_time2_ML=MLGMRES(A, b/b_norm, Out[-10], e2, nmax_iter,Err_list,IterErr0_AVG,ProbIdx,restart,debug,refine2)
    Out2=OutList2[0]
    Err_list=OutList2[1]
    IterErr0_AVG=OutList2[2]
    trainTime=OutList2[3]
    forwardTime2=OutList2[4]

    forwardTime_Total=forwardTime1+forwardTime2+forwardTime_Total
    trainTime_total=trainTime+trainTime_total
    run_time_ML=run_time1_ML+run_time2_ML+run_time_ML




    NonML_Out1,run_time1=GMRES_timed(A, b/b_norm, x0, e1, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    NonML_Out2,run_time2=GMRES_timed(A, b/b_norm, NonML_Out1[-10], e2, nmax_iter,restart, debug)  # normalize b for optimal NN performance.
    run_time=run_time1+run_time2+run_time

    print(ProbIdx)


logger.info("Runtime of Non-decorated version is:")
logger.info(run_time)

logger.info("Runtime of MLGMRES decorator is:")
logger.info(run_time_ML)


logger.info("Runtime of training (backprop) is:")
logger.info(trainTime_total)

logger.info("Runtime of forward function is:")
logger.info(forwardTime_Total)


0
1
2
loss: 39748600.0 0
loss: 39743684.0 1
loss: 39738652.0 2
loss: 39733264.0 3
loss: 39727324.0 4
loss: 39720728.0 5
loss: 39713380.0 6
loss: 39705224.0 7
loss: 39696224.0 8
loss: 39686320.0 9
loss: 39675468.0 10
loss: 39663596.0 11
loss: 39650668.0 12
loss: 39636632.0 13
loss: 39621432.0 14
loss: 39605012.0 15
loss: 39587336.0 16
loss: 39568360.0 17
loss: 39548020.0 18
loss: 39526232.0 19
loss: 39502960.0 20
loss: 39478152.0 21
loss: 39451752.0 22
loss: 39423704.0 23
loss: 39393928.0 24
loss: 39362368.0 25
loss: 39328968.0 26
loss: 39293676.0 27
loss: 39256464.0 28
loss: 39217248.0 29
loss: 39175956.0 30
loss: 39132496.0 31
loss: 39086848.0 32
loss: 39038968.0 33
loss: 38988848.0 34
loss: 38936432.0 35
loss: 38881652.0 36
loss: 38824492.0 37
loss: 38764932.0 38
loss: 38702948.0 39
loss: 38638480.0 40
loss: 38571512.0 41
loss: 38502024.0 42
loss: 38430000.0 43
loss: 38355432.0 44
loss: 38278308.0 45
loss: 38198608.0 46
loss: 38116224.0 47
loss: 38031188.0 48
loss: 37943516.0 49
loss

loss: 498381.8125 498
loss: 498086.0625 499
loss: 497796.5 500
loss: 497513.34375 501
loss: 497236.5 502
loss: 496965.625 503
loss: 496701.0 504
loss: 496441.8125 505
loss: 496188.4375 506
loss: 495940.5 507
loss: 495697.9375 508
loss: 495460.9375 509
loss: 495228.8125 510
loss: 495001.6875 511
loss: 494779.625 512
loss: 494562.375 513
loss: 494349.8125 514
loss: 494141.6875 515
loss: 493938.1875 516
loss: 493739.0625 517
loss: 493544.0625 518
loss: 493353.34375 519
loss: 493166.625 520
loss: 492983.90625 521
loss: 492805.125 522
loss: 492630.0625 523
loss: 492458.71875 524
loss: 492291.0 525
loss: 492126.84375 526
loss: 491966.125 527
loss: 491808.71875 528
loss: 491654.65625 529
loss: 491503.84375 530
loss: 491356.25 531
loss: 491211.5 532
loss: 491069.8125 533
loss: 490931.3125 534
loss: 490795.3125 535
loss: 490662.25 536
loss: 490531.90625 537
loss: 490404.15625 538
loss: 490279.125 539
loss: 490156.5625 540
loss: 490036.6875 541
loss: 489919.03125 542
loss: 489803.6875 543
loss: 

loss: 475729.8125 908
loss: 475676.9375 909
loss: 475623.71875 910
loss: 475570.40625 911
loss: 475516.6875 912
loss: 475462.75 913
loss: 475408.4375 914
loss: 475354.0625 915
loss: 475299.5 916
loss: 475244.5625 917
loss: 475189.375 918
loss: 475133.8125 919
loss: 475078.21875 920
loss: 475022.125 921
loss: 474966.03125 922
loss: 474909.65625 923
loss: 474852.8125 924
loss: 474795.6875 925
loss: 474738.40625 926
loss: 474680.8125 927
loss: 474623.0625 928
loss: 474564.75 929
loss: 474506.46875 930
loss: 474447.78125 931
loss: 474388.9375 932
loss: 474329.59375 933
loss: 474270.1875 934
loss: 474210.3125 935
loss: 474150.34375 936
loss: 474090.0 937
loss: 474029.34375 938
loss: 473968.53125 939
loss: 473907.25 940
loss: 473845.6875 941
loss: 473783.9375 942
loss: 473721.875 943
loss: 473659.4375 944
loss: 473596.90625 945
loss: 473533.9375 946
loss: 473470.59375 947
loss: 473406.96875 948
loss: 473343.3125 949
loss: 473279.03125 950
loss: 473214.59375 951
loss: 473149.875 952
loss: 473

loss: 411024.375 1314
loss: 410678.21875 1315
loss: 410330.625 1316
loss: 409981.59375 1317
loss: 409631.375 1318
loss: 409279.625 1319
loss: 408926.5 1320
loss: 408571.96875 1321
loss: 408216.125 1322
loss: 407858.75 1323
loss: 407499.90625 1324
loss: 407139.8125 1325
loss: 406778.3125 1326
loss: 406415.25 1327
loss: 406050.8125 1328
loss: 405684.9375 1329
loss: 405317.71875 1330
loss: 404948.875 1331
loss: 404578.875 1332
loss: 404207.25 1333
loss: 403834.3125 1334
loss: 403459.8125 1335
loss: 403083.875 1336
loss: 402706.5 1337
loss: 402327.75 1338
loss: 401947.53125 1339
loss: 401565.75 1340
loss: 401182.5625 1341
loss: 400798.09375 1342
loss: 400412.0 1343
loss: 400024.28125 1344
loss: 399635.1875 1345
loss: 399244.875 1346
loss: 398852.84375 1347
loss: 398459.3125 1348
loss: 398064.4375 1349
loss: 397667.875 1350
loss: 397270.0625 1351
loss: 396870.78125 1352
loss: 396469.78125 1353
loss: 396067.4375 1354
loss: 395663.625 1355
loss: 395258.125 1356
loss: 394851.46875 1357
loss: 3

loss: 173395.59375 1714
loss: 172723.46875 1715
loss: 172052.21875 1716
loss: 171382.0625 1717
loss: 170712.84375 1718
loss: 170044.578125 1719
loss: 169377.71875 1720
loss: 168711.5625 1721
loss: 168046.3125 1722
loss: 167382.421875 1723
loss: 166719.390625 1724
loss: 166057.78125 1725
loss: 165400.046875 1726
loss: 164743.328125 1727
loss: 164087.5625 1728
loss: 163433.09375 1729
loss: 162779.5625 1730
loss: 162127.34375 1731
loss: 161475.96875 1732
loss: 160825.828125 1733
loss: 160176.796875 1734
loss: 159528.9375 1735
loss: 158882.171875 1736
loss: 158236.734375 1737
loss: 157592.375 1738
loss: 156949.1875 1739
loss: 156307.46875 1740
loss: 155666.609375 1741
loss: 155027.078125 1742
loss: 154388.875 1743
loss: 153751.796875 1744
loss: 153116.03125 1745
loss: 152481.5 1746
loss: 151848.203125 1747
loss: 151216.28125 1748
loss: 150585.53125 1749
loss: 149956.15625 1750
loss: 149328.0625 1751
loss: 148701.375 1752
loss: 148075.875 1753
loss: 147451.703125 1754
loss: 146828.96875 175

...

loss: 0.016252361238002777 1428
loss: 0.015823666006326675 1429
loss: 0.016119737178087234 1430
loss: 0.016497507691383362 1431
loss: 0.016366902738809586 1432
loss: 0.015704555436968803 1433
loss: 0.014991763979196548 1434
loss: 0.014673318713903427 1435
loss: 0.014792604371905327 1436
loss: 0.014966249465942383 1437
loss: 0.014870896004140377 1438
loss: 0.014468478970229626 1439
loss: 0.014005487784743309 1440
loss: 0.013740766793489456 1441
loss: 0.013722063973546028 1442
loss: 0.013779242523014545 1443
loss: 0.013693876564502716 1444
loss: 0.01344189140945673 1445
loss: 0.013144971802830696 1446
loss: 0.012921348214149475 1447
loss: 0.012839289382100105 1448
loss: 0.012821480631828308 1449
loss: 0.012757588177919388 1450
loss: 0.012591972947120667 1451
loss: 0.012386721558868885 1452
loss: 0.012197507545351982 1453
loss: 0.012083502486348152 1454
loss: 0.01201824750751257 1455
loss: 0.011956973001360893 1456
loss: 0.01185077428817749 1457
loss: 0.011708203703165054 1458
loss: 0.011

loss: 3448.6630859375 227
loss: 3379.70068359375 228
loss: 3312.06396484375 229
loss: 3245.8857421875 230
loss: 3180.9892578125 231
loss: 3117.56591796875 232
loss: 3055.46337890625 233
loss: 2994.5439453125 234
loss: 2934.822509765625 235
loss: 2876.23046875 236
loss: 2818.8037109375 237
loss: 2762.4482421875 238
loss: 2707.20751953125 239
loss: 2653.11181640625 240
loss: 2600.10205078125 241
loss: 2548.117431640625 242
loss: 2497.13818359375 243
loss: 2447.130859375 244
loss: 2398.1865234375 245
loss: 2350.2646484375 246
loss: 2303.33056640625 247
loss: 2257.3671875 248
loss: 2212.2470703125 249
loss: 2168.00732421875 250
loss: 2124.730712890625 251
loss: 2082.421875 252
loss: 2040.975341796875 253
loss: 2000.39892578125 254
loss: 1960.630126953125 255
loss: 1921.623291015625 256
loss: 1883.408935546875 257
loss: 1845.943115234375 258
loss: 1809.265380859375 259
loss: 1773.364013671875 260
loss: 1738.16845703125 261
loss: 1703.727294921875 262
loss: 1669.977294921875 263
loss: 1636.9

loss: 18.279264450073242 532
loss: 18.01108169555664 533
loss: 17.746795654296875 534
loss: 17.488950729370117 535
loss: 17.23208236694336 536
loss: 16.97857093811035 537
loss: 16.73102378845215 538
loss: 16.48711395263672 539
loss: 16.247032165527344 540
loss: 16.010757446289062 541
loss: 15.777400970458984 542
loss: 15.54660701751709 543
loss: 15.320475578308105 544
loss: 15.09768009185791 545
loss: 14.877052307128906 546
loss: 14.661064147949219 547
loss: 14.44825267791748 548
loss: 14.238625526428223 549
loss: 14.031455993652344 550
loss: 13.828152656555176 551
loss: 13.628591537475586 552
loss: 13.430990219116211 553
loss: 13.235797882080078 554
loss: 13.044903755187988 555
loss: 12.856752395629883 556
loss: 12.671154022216797 557
loss: 12.487813949584961 558
loss: 12.308035850524902 559
loss: 12.13068675994873 560
loss: 11.95621109008789 561
loss: 11.784591674804688 562
loss: 11.614471435546875 563
loss: 11.44845962524414 564
loss: 11.28341293334961 565
loss: 11.121992111206055 5

loss: 0.4726638197898865 821
loss: 0.4677637219429016 822
loss: 0.4628753066062927 823
loss: 0.45825695991516113 824
loss: 0.453619122505188 825
loss: 0.44902944564819336 826
loss: 0.44433632493019104 827
loss: 0.43984106183052063 828
loss: 0.4353419840335846 829
loss: 0.43097811937332153 830
loss: 0.4264949560165405 831
loss: 0.4222607910633087 832
loss: 0.41801199316978455 833
loss: 0.4136973023414612 834
loss: 0.4096105694770813 835
loss: 0.40540438890457153 836
loss: 0.4013739824295044 837
loss: 0.3973884582519531 838
loss: 0.3933049142360687 839
loss: 0.3893470764160156 840
loss: 0.3854823708534241 841
loss: 0.38162845373153687 842
loss: 0.3777422308921814 843
loss: 0.37407296895980835 844
loss: 0.3703020513057709 845
loss: 0.3665207028388977 846
loss: 0.3628064692020416 847
loss: 0.3592468798160553 848
loss: 0.35565271973609924 849
loss: 0.3521782457828522 850
loss: 0.3485846519470215 851
loss: 0.3452393412590027 852
loss: 0.3417469263076782 853
loss: 0.3383581042289734 854
loss:

loss: 0.03549111634492874 1102
loss: 0.035199835896492004 1103
loss: 0.034874945878982544 1104
loss: 0.03458007425069809 1105
loss: 0.034280143678188324 1106
loss: 0.03399309515953064 1107
loss: 0.03370990604162216 1108
loss: 0.03342999517917633 1109
loss: 0.03313908725976944 1110
loss: 0.03286339342594147 1111
loss: 0.03259965404868126 1112
loss: 0.0323246568441391 1113
loss: 0.032037340104579926 1114
loss: 0.03176967799663544 1115
loss: 0.03149764984846115 1116
loss: 0.031232619658112526 1117
loss: 0.030970104038715363 1118
loss: 0.030690446496009827 1119
loss: 0.030448634177446365 1120
loss: 0.03018820844590664 1121
loss: 0.029935766011476517 1122
loss: 0.02965967170894146 1123
loss: 0.02941521257162094 1124
loss: 0.02918378636240959 1125
loss: 0.02892257645726204 1126
loss: 0.028679009526968002 1127
loss: 0.028433695435523987 1128
loss: 0.028183288872241974 1129
loss: 0.027939356863498688 1130
loss: 0.027713529765605927 1131
loss: 0.02747524157166481 1132
loss: 0.027238477021455765